<a href="https://colab.research.google.com/github/drvoss/Colab-Notebooks/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
!tar -zxvf taco_and_burrito.tar.gz

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)
import tqdm

from torchvision.datasets import ImageFolder
from torchvision import transforms

# ImageFolder 함수를 사용해서 Dataset 작성
train_imgs = ImageFolder("/content/taco_and_burrito/train/"
                         , transform=transforms.Compose([transforms.RandomCrop(224), transforms.ToTensor()]))
test_imgs = ImageFolder("/content/taco_and_burrito/test/"
                        , transform=transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()]))
# DataLoader 작성
train_loader = DataLoader(train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_imgs, batch_size=32, shuffle=False)

from torchvision import models

def eval_net(net, data_loader, device="cpu"):
  # Dropout 및 BatchNorm을 무효화
  net.eval()
  ys = []
  ypreds = []
  for x, y in data_loader:
    # to 메서드로 계산을 실행할 디바이스로 전송
    x = x.to(device)
    y = y.to(device)
    # 확률이 가장 큰 분류를 예측(리스트 2.1 참조)
    # 여기선 forward（추론） 계산이 전부이므로 자동 미분에
    # 필요한 처리는 off로 설정해서 불필요한 계산을 제한다
    with torch.no_grad():
      _, y_pred = net(x).max(1)
    ys.append(y)
    ypreds.append(y_pred)
    
  # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  # 예측 정확도 계산
  acc = (ys == ypreds).float().sum() / len(ys)
  return acc.item()

def train_net(net, train_loader, test_loader,only_fc=True
              , optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss()
              , n_iter=10, device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  if only_fc:
    # 마지막 선형 계층의 파라미터만
    # optimizer에 전달
    optimizer = optimizer_cls(net.fc.parameters())
  else:
    optimizer = optimizer_cls(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    # 신경망을 훈련 모드로 설정
    net.train()
    n = 0
    n_acc = 0
    # 시간이 많이 걸리므로 tqdm을 사용해서 진행 바를 표시
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
      total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      h = net(xx)
      loss = loss_fn(h, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
      _, y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()
    train_losses.append(running_loss / i)
    # 훈련 데이터의 예측 정확도
    train_acc.append(n_acc / n)
    
    # 검증 데이터의 예측 정확도
    val_acc.append(eval_net(net, test_loader, device))
    # epoch의 결과 표시
    print(epoch, train_losses[-1], train_acc[-1],
      val_acc[-1], flush=True)

"""
# 사전 학습이 완료된 resnet18 불러오기
net = models.resnet18(pretrained=True)
# 모든 파라미터를 미분 대상에서 제외한다
for p in net.parameters():
  p.requires_grad=False
# 마지막 선형 계층을 변경한다
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)
# 신경망의 모든 파라미터를 GPU로 전송
net.to("cuda:0")
# 훈련 실행
train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")
"""
# (N, C, H, W)형식의 텐서를(N, C*H*W)로 늘리는 계층
# 합성곱 출력을 MLP에 전달할 때 필요
class FlattenLayer(nn.Module):
  def forward(self, x):
    sizes = x.size()
    return x.view(sizes[0], -1)

class IdentityLayer(nn.Module):
  def forward(self, x):
    return x
  
net = models.resnet18(pretrained=True)
for p in net.parameters():
  p.requires_grad=False
net.fc = IdentityLayer()

conv_net = nn.Sequential(
  nn.Conv2d(3, 32, 5),
  nn.MaxPool2d(2),
  nn.ReLU(),
  nn.BatchNorm2d(32),
  nn.Conv2d(32, 64, 5),
  nn.MaxPool2d(2),
  nn.ReLU(),
  nn.BatchNorm2d(64),
  nn.Conv2d(64, 128, 5),
  nn.MaxPool2d(2),
  nn.ReLU(),
  nn.BatchNorm2d(128),
  FlattenLayer()
)
# 합성곱에 의해 최종적으로 어떤 크기인지
# 실제로 데이터를 넣어서 확인
test_input = torch.ones(1, 3, 224, 224)
conv_output_size = conv_net(test_input).size()[-1]
# 최종 CNNN
net = nn.Sequential(
  conv_net,
  nn.Linear(conv_output_size, 2)
)

# 신경망의 모든 파라미터를 GPU로 전송
net.to("cuda:0")

# 훈련 실행
train_net(net, train_loader, test_loader, n_iter=10, only_fc=False, device="cuda:0")

100%|██████████| 23/23 [00:06<00:00,  3.74it/s]


0 2.708134201439944 0.550561797752809 0.4833333492279053


100%|██████████| 23/23 [00:05<00:00,  3.88it/s]


1 2.346039907498793 0.6095505617977528 0.5333333611488342


100%|██████████| 23/23 [00:05<00:00,  4.59it/s]


...

7 1.9943181682716717 0.6769662921348315 0.5666667222976685


100%|██████████| 23/23 [00:06<00:00,  3.83it/s]


8 2.1651138988408176 0.6193820224719101 0.6833333969116211


100%|██████████| 23/23 [00:05<00:00,  3.85it/s]


9 2.6410754214633596 0.6587078651685393 0.6666666865348816
